In [1]:
# import sys
# !{sys.executable} -m pip install 
# import gdal
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import glacml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import re
print(tf.__version__)

# Make numpy printouts easier to read.
np.set_printoptions(precision=3, suppress=True)

# raw_dataset = pd.read_csv("/data/fast0/datasets/glathida-3.1.0/data/T.csv")
# raw_dataset = pd.read_csv("~/stuff/coding/glacier/data/T.csv")


#examine data columns
# dataset.head()

2.7.0


In [38]:
raw_dataset = pd.read_csv("~/data/glac/T_models/T.csv")
glathida = raw_dataset.copy()
glathida = glathida.drop([
#                          "GLACIER_DB",
                         "POLITICAL_UNIT",
                         "GLACIER_NAME",
                         "SURVEY_DATE",
                         "MEAN_THICKNESS_UNCERTAINTY",
#                          "MAXIMUM_THICKNESS",
                         "MAX_THICKNESS_UNCERTAINTY",
                         "DATA_FLAG",
                         "ELEVATION_DATE",
                         "SPONSORING_AGENCY",
                         "REMARKS",
                         "SURVEY_METHOD_DETAILS",
                         "SURVEY_METHOD",
                         "NUMBER_OF_SURVEY_POINTS",
                         "NUMBER_OF_SURVEY_PROFILES",
                         "TOTAL_LENGTH_OF_SURVEY_PROFILES",
                         "INTERPOLATION_METHOD",
                         "INVESTIGATOR",
                         "REFERENCES",
#                          "GLACIER_ID",
                         "GlaThiDa_ID",
                        ],axis=1)

glathida.rename(columns = {
                      "LAT":"LAT_G",
                      "LON":"LON_G",
                      "AREA":"AREA_G",
                      "MEAN_SLOPE":"MEAN_SLOPE_G"
                     },
           inplace = True)
glathida = glathida.dropna()

with open("/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv",'r') as temp_f:
# get No of columns in each line
    col_count = [ len(l.split(",")) for l in temp_f.readlines() ]

### Generate column names  (names will be 0, 1, 2, ..., maximum columns - 1)
column_names = [i for i in range(0, max(col_count))]

### Read csv
df = pd.read_csv("/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv",
                 header=None, delimiter=",",names=column_names)
col_names = list(df.iloc[0])
df = pd.read_csv("/home/sa42/data/glac/glims/glims_extra/dump2/13_rgi60_CentralAsia.csv",
                 header=None, delimiter=",", names=col_names)
df.drop(0, inplace=True); df.reset_index(drop=True, inplace=True)
RGI = df

idx = glathida.index[glathida["GLACIER_DB"]=="RGI"]
glathida = glathida.loc[idx]
glathida.rename(columns = {"GLACIER_ID":"RGIId"},inplace=True)

RGI_glathida = pd.merge(glathida,RGI, how = "inner", on="RGIId")
RGI_glathida = RGI_glathida.drop([
                                  "GLACIER_DB",
                                  "RGIId",
                                  "GLIMSId",
                                  "BgnDate",
                                  "Status",
                                  "Connect",
                                  "TermType",
                                  "Surging",
                                  "Linkages",
                                  "Name",
                                  "O1Region",
                                  "O2Region",
                                  "EndDate",
                                  "Name"
                                 ],axis=1)
#split the dataset and reserve some to test what was trained.
# train_dataset = RGI_glathida.sample(frac=0.8, random_state=0)
# test_dataset = RGI_glathida.drop(train_dataset.index)


In [26]:
raw_dataset = pd.read_csv("~/data/glac/T_models/T.csv")
T_dataset = raw_dataset
T_dataset = T_dataset.drop("GLACIER_DB", axis=1)
T_dataset = T_dataset.drop("POLITICAL_UNIT", axis=1)
T_dataset = T_dataset.drop("GLACIER_ID", axis=1)
T_dataset = T_dataset.drop("GLACIER_NAME", axis=1)
T_dataset = T_dataset.drop("NUMBER_OF_SURVEY_POINTS", axis=1)
T_dataset = T_dataset.drop("NUMBER_OF_SURVEY_PROFILES", axis=1)
T_dataset = T_dataset.drop("TOTAL_LENGTH_OF_SURVEY_PROFILES", axis=1)
T_dataset = T_dataset.drop("INTERPOLATION_METHOD", axis=1)
T_dataset = T_dataset.drop("INVESTIGATOR", axis=1)
T_dataset = T_dataset.drop("SPONSORING_AGENCY", axis=1)
T_dataset = T_dataset.drop("ELEVATION_DATE", axis=1)
T_dataset = T_dataset.drop("SURVEY_METHOD_DETAILS", axis=1)
T_dataset = T_dataset.drop("SURVEY_METHOD", axis=1)
T_dataset = T_dataset.drop("SURVEY_DATE", axis=1)
T_dataset = T_dataset.drop("MEAN_THICKNESS", axis=1)
T_dataset = T_dataset.drop("AREA", axis=1)
T_dataset = T_dataset.drop("MEAN_THICKNESS_UNCERTAINTY", axis=1)
T_dataset = T_dataset.drop("MAXIMUM_THICKNESS", axis=1)
T_dataset = T_dataset.drop("MAX_THICKNESS_UNCERTAINTY", axis=1)
T_dataset = T_dataset.drop("DATA_FLAG", axis=1)
T_dataset = T_dataset.drop("REMARKS", axis=1)
T_dataset = T_dataset.drop("REFERENCES", axis=1)
T_dataset = T_dataset.drop("MEAN_SLOPE", axis=1)
T_dataset = T_dataset.dropna()
T_dataset

,GlaThiDa_ID,LAT,LON
0,1,67.91500,18.56800
1,2,67.91000,18.49600
2,3,67.90000,18.57000
3,4,48.35698,-121.05735
4,5,52.17540,-117.28400
...,...,...,...
5136,6627,46.20240,-121.49090
5137,6628,46.20240,-121.49090
5138,6629,46.20398,-121.49376
5139,6630,47.08670,12.38000


In [31]:
train_dataset.describe()

,LAT_G,LON_G,AREA_G,MEAN_SLOPE_G,MEAN_THICKNESS,MAXIMUM_THICKNESS,NaN
count,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,0.0
mean,41.877250,88.130500,2.353870,19.750000,29.500000,115.750000,NaN
std,2.614125,8.562494,2.519038,6.849574,20.041623,55.319526,NaN
min,38.214000,79.894000,0.536800,10.000000,15.000000,66.000000,NaN
25%,40.888500,83.266750,1.069082,17.500000,15.750000,70.500000,NaN
50%,42.755500,86.373500,1.400340,22.000000,22.500000,110.000000,NaN
75%,43.744250,91.237250,2.685128,24.250000,36.250000,155.250000,NaN
max,43.784000,99.881000,6.078000,25.000000,58.000000,177.000000,NaN


In [ ]:
#separate out features - what will be trained to predict desired attribute
train_features = train_dataset.copy()
test_features = test_dataset.copy()

#define label - attribute training to be picked
train_labels = train_features.pop("MEAN_THICKNESS")
test_labels = test_features.pop("MEAN_THICKNESS")

# train_features.describe().transpose()[['mean', 'std']]

# Establish normalization layers

In [ ]:
normalizer = {}
variable_list = list(train_features)
for variable_name in variable_list:

    normalizer[variable_name] = preprocessing.Normalization(input_shape=[1,], axis=None)
    normalizer[variable_name].adapt(np.array(train_features[variable_name]))
    
    
normalizer['ALL'] = preprocessing.Normalization(axis=-1)
normalizer['ALL'].adapt(np.array(train_features))

In [ ]:
linear_model = {}
linear_history = {}
linear_results = {}

In [ ]:
%%time
for variable_name in variable_list:

    linear_model[variable_name] = gl.build_linear_model(normalizer[variable_name])
    linear_history[variable_name] = linear_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    
    
    linear_results[variable_name] = linear_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

linear_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(linear_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print("")


In [ ]:
# import glacml as gl
# gl.plot_loss(linear_history)

# Linear regression with multiple inputs

In [ ]:
linear_model = gl.build_linear_model(normalizer['ALL'])

history_full = linear_model.fit(
train_features, train_labels,        
   epochs=1000,
   verbose=0,
   validation_split = 0.2)

test_results["MULTI"] = linear_model.evaluate(
    test_features,
    test_labels, verbose=0)

test_results

In [ ]:
df = pd.DataFrame(history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)



In [ ]:
plot_loss(history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_full_loss.eps")

# DNN regression

## Define regression functions

In [ ]:
def build_dnn_model(norm):
    model = keras.Sequential([
              norm,
              layers.Dense(64, activation='relu'),
              layers.Dense(64, activation='relu'),
              layers.Dense(1) ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.01))
    
    return model

def dnn_test_results(feature_name):
    dnn_test_results[feature_name] = dnn_model.evaluate(
        test_features[feature_name],
        test_labels, verbose=0)

In [ ]:
dnn_model = {}
dnn_history = {}
dnn_results = {}

In [ ]:
for variable_name in variable_list:

    dnn_model[variable_name] = build_dnn_model(normalizer[variable_name])
    dnn_history[variable_name] = dnn_model[variable_name].fit(
                                        train_features[variable_name], train_labels,        
                                        epochs=1000,
                                        verbose=0,
                                        validation_split = 0.2)
    dnn_results[variable_name_dnn] = dnn_model[variable_name].evaluate(
                                        test_features[variable_name],
                                        test_labels, verbose=0)

dnn_results

In [ ]:
for variable_name in variable_list:    
    df = pd.DataFrame(dnn_history[variable_name].history)
    dfs = df.loc[[df.last_valid_index()]]
    dfs.insert(0, 'Variable', [variable_name])
    
    print(dfs)
    print("")


In [ ]:
fig,ax=plt.subplots(2,2,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    plt.subplot(2,2,i+1)
    xmax = np.max(train_features[variable_name])
    xmin = np.min(train_features[variable_name])
    x = tf.linspace(xmin, xmax, 101)
    y = dnn_model[variable_name].predict(x)
    plot_single_model_variable(x,y,variable_name)

In [ ]:
fig,ax=plt.subplots(2,2,figsize=(10,10))
for i, variable_name in enumerate(variable_list):
    ax = plt.subplot(2,2,i+1)
    plot_loss(dnn_history[variable_name])
    ax.set_title(variable_name)
#     plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_loss.eps")

# Full model

In [ ]:
dnn_full_model = build_dnn_model(normalizer['ALL'])

In [ ]:
%%time
dnn_history_full = dnn_full_model.fit(
    train_features, train_labels,
    validation_split=0.2,
    verbose=0, epochs=1000)

In [ ]:
df = pd.DataFrame(dnn_history_full.history)
dfs = df.loc[[df.last_valid_index()]]
dfs.insert(0, 'Variable', 'Multi-Variable')
    
print(dfs)


In [ ]:
plot_loss(dnn_history_full)
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_dnn_full_loss.png")

In [ ]:
y = dnn_full_model.predict(test_features)
fig,ax=plt.subplots()
fig.patch.set_facecolor('w')
plt.plot(test_labels,y,"o")
plt.plot((0,120),(0,120),'-')
plt.xlabel('True Thickness (m)')
plt.ylabel('Model Thickness (m)')
plt.xlim((0,120))
plt.ylim((0,120))
# plt.savefig("/home/sa42/notebooks/glac/figs/GTP1_res.EPS")